In [71]:
import requests, re, json
from datetime import datetime
class LLMClient:
    def __init__(self, api_key):
        self._url = 'https://np6jbwtoaiv4y8cw.us-east4.gcp.endpoints.huggingface.cloud/v1/chat/completions'
        self._api_key = api_key

    def make_chat_completions_request(self, model, messages, temperature, max_tokens):
        headers = {
            "Authorization": f"Bearer {self._api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stream": False,
        }
        response = requests.post(self._url, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        return response.json()

In [90]:
res = requests.post(
        "https://www.medicare.gov/api/care-compare/provider",
        json={
            "type": "Physician",
            "filters": {
                "radiusSearch": {
                    "coordinates": {
                        "lon": -122.09444,
                        "lat": 37.29426,
                    },
                    "radius": 15,
                }
            },
            "returnAllResults": True,
            "sort": ["closest"],
        },
    )

In [ ]:
res = json.loads(res.text)
print(json.dumps(res, indent=2))

In [31]:
llmClient = LLMClient("")
def get_response(name, date_time_range, insurance_company, policy_num):
    messages = [{'role': 'user', 'content': 
        f"""
        You are an assistant from CDF company that calls the clinic to ask for available slots and confirm the insurance coverage.
        Here is the date and time range you need to check:
        {date_time_range}
        Here is the patient's name:
        {name}
        Insurance Company:
        {insurance_company}
        Policy number:
        {policy_num}
        Now, I will call the clinic for you. Wait fot my response.
        """+"""
If you get all the information of you need, please respond in this format:
END
```
{
    "available_time_slots": [...],
    "is_in_network": true/false
}
```
Otherwise, please respond in this format
```
{
    "response": "..."
}
```
"""
    },
    {'role': 'assistant', 'content': 
    """
    Yeah! I can help you with that.
    """
    },
    ]
    
    info  = {}
    # wait for clinic response 
    clinic_response = input("This is the first sentence when you pick up a phone")
    messages.append({'role': 'user', 'content': build_user_prompt(clinic_response=clinic_response)})
    phone_connected = True
    while(phone_connected):
        assistant_response = llmClient.make_chat_completions_request(
            model="27b-text-it",
            messages=messages,
            temperature=0.7,
            max_tokens=200
        )
        print(assistant_response)
        messages.append(assistant_response["choices"][0]["message"])
        assistant_message = assistant_response["choices"][0]["message"]["content"]
        if "END" in assistant_message:
            phone_connected = False
            info = parse_markdown_json(assistant_message)
            break
        # send assistant_message to clinic and wait for clinic response 
        clinic_response = input(assistant_message)

        if clinic_response.lower() == "n":
            phone_connected = False
            messages.append(
                {'role': 'user', 'content': 
                """
                The phone call is ended. Please return the information you have now.
                available_time_slots default is empty list
                is_in_network default is false
                reply in this format:
                ```
                {
                    "available_time_slots": [...],
                    "is_in_network": true/false
                }
                ```
                """
                }
            )
            assistant_response = llmClient.make_chat_completions_request(
            model="27b-text-it",
            messages=messages,
            temperature=0.7,
            max_tokens=200
        )
            assistant_message = assistant_response["choices"][0]["message"]["content"]
            info = parse_markdown_json(assistant_message)
            break
        else:
            messages.append(
                {'role': 'user', 'content': build_user_prompt(clinic_response=clinic_response)}
            )
    return info


def parse_markdown_json(markdown_str):
    # Regex to match JSON object within triple backticks
    pattern = r'```(json)?[\n\s]*({.*?})[\n\s]*```'
    
    # Find the JSON object
    match = re.search(pattern, markdown_str, re.DOTALL)
    if not match:
        return {"error": "No JSON object found in markdown"}
    
    # Extract and parse the JSON string
    json_str = match.group(2)
    try:
        data = json.loads(json_str)
        return data
    except json.JSONDecodeError as e:
        return {"error": f"Invalid JSON: {str(e)}"}
    except KeyError as e:
        return {"error": f"Missing key: {str(e)}"}
    
def build_user_prompt(clinic_response):
    return """
The clinic says:
\n""" + clinic_response+"""
====
What do you want to respond? 
If you get all the information of you need, please respond in this format:
END
```
{
    "available_time_slots": [...],
    "is_in_network": true/false
}
```

Otherwise, please respond in this format
```
{
    "response": ...
}
```
"""

In [ ]:
requests.get()

In [ ]:
get_response("Jeff Liu", "2025-05-15 14:00-15:00", "cigna", "123456789")

In [34]:
import requests
call_id=""
api_key=""
url = f"https://api.bland.ai/v1/calls/{call_id}"
headers = {"authorization": api_key}
response = requests.get(url, headers=headers)

In [41]:
with open("transcript.json", "w") as f:
    res = json.loads(response.text)
    json.dump(res, f, indent=2)

In [ ]:
transcript = ""
for conversation in res["pathway_logs"]:
    if conversation["role"] in set(["user", "assistant"]):
        transcript += f"{conversation['role']}: {conversation['text']}\n"

print(transcript)


In [7]:
import requests

url = 'https://api.bland.ai/v1/calls'
headers = {'authorization': 'org_3753f3fdbeba2861d59528ce6c00d79a3809334ca2cfec8a27ff598411d19aab20acc0c0144e5afc2d0769'}
data = {
    'phone_number': '+16507884580',
    'pathway_id': '9d59bc86-ede8-4ecc-b276-e72f50c47353',
    'request_data': {'policy_num': '7G91HP2Q3R4', 
                  'insurance_company': 'cigna', 
                  'date_time_range': "05-20 14:00-16:00", 
                  'name': 'Jeff Liu',
                  'provider_name': 'Dr. Nikhil Krishnan',
                  },
}

response = requests.post(url, json=data, headers=headers)
print('Call initiated:', response.json()['call_id'])

Call initiated: be52e1ef-1714-4194-8070-d89fe8a6ae2f


In [ ]:
import requests

url = 'https://api.bland.ai/v1/calls'
headers = {'authorization': ''}
data = {
    'phone_number': '',
    'pathway_id': '',
    'request_data': {
        'policy_num': '1111-456-789-00', 
        'insurance_company': 'cigna', 
        'date_time_range': "05-18 14:00 to 16:00, 05-19 09:00 to 12:00", 
        'name': 'Jeff Liu',
    },
}

response = requests.post(url, json=data, headers=headers)
print('Call initiated:', response.json()['call_id'])

In [73]:
res = llmClient.make_chat_completions_request(
    model="27b-text-it",
    messages=[{"role":"user", "content":
f"""
Here is the transcription of assistant and user
User is clinic side
today is {datetime.now().strftime("%Y-%m-%d")}
transcription
{transcript}
"""+
"""
====
24hr timeslot format: YYYY-MM-DD HH:MM - HH:MM
Please extract the below information
```
{
    'available_timeslot': [...],
    'is_in_nework: true/false
}
```
"""}],
            temperature=0.7,
            max_tokens=200
)


In [ ]:
parse_markdown_json(res["choices"][0]["message"]["content"])


In [55]:
res = requests.get("http://localhost:8000/providers/recommend", 
                json={
                    "zipCode": 95014,
                    "radius": 50,
                        "symptomDescription": "hard to eat. hard to sleep",
                    "patientInfo":{
    "insuranceCompany": "cigna",
    "policyNum": "123456789",
    "dateTimeRange": "2025-05-15 14:00-15:00",
    "name": "Jeff Liu",
    "dateOfBirth": "1999-11-01",
}})

In [56]:
import json
print(json.dumps(json.loads(res.text), indent=2))

[
  {
    "provider_infos": [
      {
        "distance": 0.704,
        "name": "Tsung-Chang Chen, MD",
        "id": "42079e0c-641b-4ac3-9f2a-71674afa5770",
        "type": "Physician",
        "providerId": "1669546651",
        "sortName": "Chen Tsung-Chang MD",
        "firstName": "Tsung-Chang",
        "lastName": "Chen",
        "addressState": "CA",
        "isDuplicateAddress": false,
        "isCanonicalAddress": false,
        "sort_name": "Chen Tsung-Chang MD",
        "physician": {
          "name": "Tsung-Chang Chen, MD",
          "dateImported": "2025-05-06T21:36:41.855Z",
          "hash": "a5836d0691d7f113977be8e82f930616",
          "id": "42079e0c-641b-4ac3-9f2a-71674afa5770",
          "providerId": "1669546651",
          "addressLine1": "10430 S DE Anza Blvd",
          "addressCity": "Cupertino",
          "addressState": "CA",
          "addressZipcode": "950143022",
          "phone": "4082578860",
          "geocodeSource": "smartystreets_strict",
         

In [54]:
res.text

'Internal Server Error'